In [1]:
from transformers import AutoTokenizer, LxmertForQuestionAnswering
from modelling_frcnn import GeneralizedRCNN
from processing_image import Preprocess
from utils import Config
from pathlib import Path
import glob
import torch
import json
from PIL import Image
import torchvision
import cv2

c:\Users\opoob\miniconda3\envs\cse493\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained('unc-nlp/lxmert-vqa-uncased')
model = LxmertForQuestionAnswering.from_pretrained('unc-nlp/lxmert-vqa-uncased')

frcnn_cfg = Config.from_pretrained('unc-nlp/frcnn-vg-finetuned') # loads heads, hidden layer data, etc. 
frcnn = GeneralizedRCNN.from_pretrained('unc-nlp/frcnn-vg-finetuned', config=frcnn_cfg)
image_preprocess = Preprocess(frcnn_cfg)

loading configuration file cache
loading weights file https://cdn.huggingface.co/unc-nlp/frcnn-vg-finetuned/pytorch_model.bin from cache at C:\Users\opoob/.cache\torch\transformers\57f6df6abe353be2773f2700159c65615babf39ab5b48114d2b49267672ae10f.77b59256a4cf8343ae0f923246a81489fc8d82f98d082edc2d2037c977c0d9d0
All model checkpoint weights were used when initializing GeneralizedRCNN.

All the weights of GeneralizedRCNN were initialized from the model checkpoint at unc-nlp/frcnn-vg-finetuned.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GeneralizedRCNN for predictions without further training.


In [3]:
# Language features
# https://huggingface.co/docs/transformers/main_classes/tokenizer
input_questions = json.load(open('../Questions/v3_OpenEnded_mscoco_val2014_questions.json'))
input_questions = [q['question'] for q in input_questions['questions']]
tokens = tokenizer(
    input_questions, 
    padding="max_length", 
    max_length=20, 
    truncation=True, 
    return_token_type_ids=True, # if multiple phrases together
    return_attention_mask=True, # for padding
    add_special_tokens=True,
    return_tensors='pt')
inputs = tokens['input_ids']

In [3]:
# Visual features ROI pooled bounding boxes

# process images into tensors and preprocess them
path = '../Dataset/v3_val2014_gen/'
transform = torchvision.transforms.Compose([torchvision.transforms.PILToTensor()])
imgs = []
count = 0
for filename in glob.glob(f'{path}/*.png'):
    im = Image.open(filename)
    temp = transform(im).movedim(0, 2)
    if count % 50 == 0:
        cv2.imshow('Image', cv2.cvtColor(temp.numpy(), cv2.COLOR_RGB2BGR))
        cv2.waitKey(0)
    imgs.append(temp)
    count += 1
cv2.destroyAllWindows()
images, sizes, scales_yx = image_preprocess(imgs[0])

# sizes is tensor of image sizes
# scales_yx is how much each image scaled by to the max image size
# images are adjusted images
output_dict = frcnn(
    images, 
    sizes,
    scales_yx=scales_yx, 
    padding='max_detections',
    max_detections=frcnn_cfg.max_detections,
    return_tensors="pt"
)

c:\Users\opoob\miniconda3\envs\cse493\lib\site-packages\torch\functional.py:478: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\b\abs_bao0hdcrdh\croot\pytorch_1675190257512\work\aten\src\ATen\native\TensorShape.cpp:2895.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


: 